# Analyzing Walt Disney World Waits - Data Cleaning

## Importing Our Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
sns.set_style('darkgrid')
init_notebook_mode
cf.go_offline
%matplotlib inline

In [2]:
# Read in the data
dwarfs = pd.read_csv('7_dwarfs_train.csv')
aliens = pd.read_csv('alien_saucers.csv')
dinosaur = pd.read_csv('dinosaur.csv')
everest = pd.read_csv('expedition_everest.csv')
fop = pd.read_csv('flight_of_passage.csv')
safari = pd.read_csv('kilimanjaro_safaris.csv')
river = pd.read_csv('navi_river.csv')
pirates = pd.read_csv('pirates_of_caribbean.csv')
rockcoaster = pd.read_csv('rock_n_rollercoaster.csv')
slinky = pd.read_csv('slinky_dog.csv')
soarin = pd.read_csv('soarin.csv')
spaceship = pd.read_csv('spaceship_earth.csv')
splash = pd.read_csv('splash_mountain.csv')
toystory = pd.read_csv('toy_story_mania.csv')

meta = pd.read_csv('metadata.csv')
entities = pd.read_csv('entities.csv')

In [3]:
# Split the rides and their names into parks
mk = [dwarfs, pirates, splash]
mk_names = ['Seven Dwarfs Mine Train', 'Pirates of the Caribbean', 'Splash Mountain']

ep = [soarin, spaceship]
ep_names = ['Soarin', 'Spaceship Earth']

hs = [aliens, rockcoaster, slinky, toystory]
hs_names = ['Alien Swirling Saucers', 'Rock-n-Rollercoaster', 'Slinky Dog Dash', 'Toy Story Midway Mania']

ak = [everest, fop, safari, river]
ak_names = ['Expedition Everest', 'Avatar Flight of Passage', 'Kilimanjaro Safaris', 'Navi River Journey']

all_rides = [dwarfs, pirates, splash, soarin, spaceship, aliens, rockcoaster, slinky,
             toystory, everest, fop, safari, river]
all_ride_names = ['Seven Dwarfs Mine Train', 'Pirates of the Caribbean', 'Splash Mountain', 'Soarin', 'Spaceship Earth',
                 'Alien Swirling Saucers', 'Rock-n-Rollercoaster', 'Slinky Dog Dash', 'Toy Story Midway Mania',
                 'Expedition Everest', 'Avatar Flight of Passage', 'Kilimanjaro Safaris', 'Navi River Journey']

# Data Cleaning

First, let's get an idea of what each ride's table looks like, using Splash Mountain as an example.

In [4]:
splash.head()

,date,datetime,SACTMIN,SPOSTMIN
0,01/01/2015,2015-01-01 07:51:12,NaN,5.0
1,01/01/2015,2015-01-01 08:02:13,NaN,5.0
2,01/01/2015,2015-01-01 08:09:12,NaN,5.0
3,01/01/2015,2015-01-01 08:16:12,NaN,5.0
4,01/01/2015,2015-01-01 08:23:12,NaN,5.0


In [5]:
splash.tail()

,date,datetime,SACTMIN,SPOSTMIN
287943,12/28/2021,2021-12-28 22:35:34,NaN,15.0
287944,12/28/2021,2021-12-28 22:42:15,NaN,15.0
287945,12/28/2021,2021-12-28 22:48:12,NaN,15.0
287946,12/28/2021,2021-12-28 22:54:10,NaN,15.0
287947,12/28/2021,2021-12-28 23:00:27,NaN,15.0


In [6]:
splash.shape

(287948, 4)

Let's first check that the date columns are properly formatted; for our visualization purposes at least, we do want them as datetime objects.

In [7]:
splash.dtypes

date         object
datetime     object
SACTMIN     float64
SPOSTMIN    float64
dtype: object

In [8]:
def fix_columns(lst):
    for ride in lst:
        ride['date'] = pd.to_datetime(ride['date'])
        ride['datetime'] = pd.to_datetime(ride['datetime'])
    return lst

fix_columns(all_rides)

print(splash.dtypes)

date        datetime64[ns]
datetime    datetime64[ns]
SACTMIN            float64
SPOSTMIN           float64
dtype: object


In [9]:
splash['datetime'].min()

Timestamp('2015-01-01 07:51:12')

In [10]:
splash['datetime'].max()

Timestamp('2021-12-28 23:00:27')

The Touring Plans data sets are updated regularly; the dates currently range from January 1, 2015 (some rides first opened at later dates) to December 28, 2021.
Actual wait times (`SACTMIN`) provided by real guests have been included for instances where an officially posted wait time (`SPOSTMIN`) was unavailable - because the official times are sometimes inflated, we'll use `SACTMIN` wherever possible. Where `SACTMIN` is not provided, we can use `SPOSTMIN` instead.

Let's create a function that will combine this data into a single column, `wait`, and then check that it worked correctly.

In [11]:
def combine_waits(lst):
    for ride in lst:
        ride['wait'] = np.where(ride['SACTMIN'].isnull(), ride['SPOSTMIN'], ride['SACTMIN'])
        ride.drop(['SACTMIN', 'SPOSTMIN'], axis=1, inplace=True)
    return lst
    
combine_waits(all_rides)

for ride in all_rides:
    print(ride.head(1))

        date            datetime  wait
0 2015-01-01 2015-01-01 07:51:12  45.0
        date            datetime   wait
0 2015-01-01 2015-01-01 08:27:58 -999.0
        date            datetime  wait
0 2015-01-01 2015-01-01 07:51:12   5.0
        date            datetime  wait
0 2015-01-01 2015-01-01 07:45:15  10.0
        date            datetime  wait
0 2015-01-01 2015-01-01 07:45:15   5.0
        date            datetime   wait
0 2018-06-30 2018-06-30 07:44:06  150.0
        date            datetime  wait
0 2015-01-01 2015-01-01 07:46:22   5.0
        date            datetime   wait
0 2018-06-30 2018-06-30 07:44:06  270.0
        date            datetime  wait
0 2015-01-01 2015-01-01 07:51:16  20.0
        date            datetime  wait
0 2015-01-01 2015-01-01 07:47:26   5.0
        date            datetime   wait
0 2017-05-26 2017-05-26 09:06:38 -999.0
        date            datetime  wait
0 2015-01-01 2015-01-01 07:47:26   5.0
        date            datetime   wait
0 2017-05-26 201

Note how some wait times are marked `-999`: are these anomalies to be fixed, or do they mean something else? Let's make a table of information from each ride and see what the general wait numbers look like.

In [12]:
def desc_table(all_ride_names, all_rides):
    df_list = []
    for ride_name, ride in zip(all_ride_names, all_rides):
               df = pd.DataFrame(ride['wait'].describe().tolist(), columns=[ride_name])
               df_list.append(df)
    wdw_desc = pd.concat(df_list, axis=1)
    wdw_desc['measures'] = ['count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max']
    first_column = wdw_desc.pop('measures')
    wdw_desc.insert(0, 'measures', first_column)
    return wdw_desc

desc_table(all_ride_names, all_rides)

,measures,Seven Dwarfs Mine Train,Pirates of the Caribbean,Splash Mountain,Soarin,Spaceship Earth,Alien Swirling Saucers,Rock-n-Rollercoaster,Slinky Dog Dash,Toy Story Midway Mania,Expedition Everest,Avatar Flight of Passage,Kilimanjaro Safaris,Navi River Journey
0,count,321631.000000,301946.000000,287948.000000,274770.000000,277248.000000,129876.000000,277509.000000,135946.000000,284170.000000,275274.000000,184818.000000,257785.000000,182121.000000
1,mean,-1.445026,-46.450296,-72.723148,24.830305,-28.370546,-10.506306,1.122032,-12.724847,18.030271,-43.429161,81.357719,-17.046989,17.565706
2,std,323.803268,267.035552,328.978529,146.128720,214.079409,199.908487,241.088011,288.941029,256.327244,269.107634,227.025416,237.237632,214.031744
3,min,-92918.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000,-999.000000
4,25%,45.000000,10.000000,10.000000,25.000000,5.000000,15.000000,30.000000,50.000000,30.000000,10.000000,70.000000,10.000000,35.000000
5,50%,70.000000,25.000000,35.000000,40.000000,15.000000,30.000000,50.000000,70.000000,50.000000,25.000000,110.000000,30.000000,60.000000
6,75%,90.000000,40.000000,60.000000,60.000000,25.000000,40.000000,75.000000,85.000000,70.000000,40.000000,150.000000,55.000000,80.000000
7,max,300.000000,185.000000,300.000000,240.000000,155.000000,190.000000,300.000000,305.000000,90387.000000,180.000000,47897.000000,300.000000,225.000000


The negative numbers prevalent throughout the ride tables and are numerous enough to throw off some of the numbers substantially.

They are initially confusing but have a good explanation: According to the dataset [documentation](https://cdn.touringplans.com/datasets/touringplans_data_dictionary.xlsx), a wait time value of `-999` means the attraction was temporarily closed or not running at the time the wait was queried. We'll have to account for this in our analysis; my initial plan is to either remove these downtimes entirely or sequester them to a separate variable. Let's see how many there are:

In [13]:
down_counts = []
for ride_name, ride in zip(all_ride_names, all_rides):
    down = [ride_name, ride['wait'].value_counts().loc[-999], (len(ride[ride['wait'] == -999]) / len(ride)) * 100]
    down_counts.append(down)
    
down_counts = pd.DataFrame(down_counts, columns=['Ride', 'Down Counts', 'Pct'])
down_counts.sort_values(by='Pct', ascending=False)

,Ride,Down Counts,Pct
2,Splash Mountain,32038,11.126314
7,Slinky Dog Dash,10657,7.839142
9,Expedition Everest,20102,7.302542
1,Pirates of the Caribbean,21915,7.257920
0,Seven Dwarfs Mine Train,23062,7.170329
11,Kilimanjaro Safaris,14033,5.443684
6,Rock-n-Rollercoaster,15001,5.405590
4,Spaceship Earth,12807,4.619330
12,Navi River Journey,7567,4.154930
5,Alien Swirling Saucers,5073,3.906033


It looks like Splash Mountain has the most posted closures, and Soarin' the least. The data dictionary is unclear as to whether Splash's (and other rides') numerous downtimes have to do with just temporary closures or also lengthy closures for refurbishments.

We again referenced Touring Plans for information about closures for refurbishments (where rides would be closed for weeks or even months at a time, not just a few hours):
* Splash Mountain: Closed for refurbishments from 01/05/2015 — 01/31/2015, 01/10/2016 — 01/14/2016, 12/07/2016 — 12/08/2016, 08/28/2017 — 11/16/2017, 01/08/2018 — 02/01/2018, and 01/06/2020 — 02/27/2020
* Pirates of the Caribbean: Closed for refurbishments from 06/08/2015 — 09/25/2015 and 02/26/2018 — 03/18/2018
* Rock'n'Rollercoaster: Closed for refurbishments from 10/27/2016 — 11/09/2016
* Toy Story Midway Mania: One-day closure 03/15/2016 — 03/16/2016, closed for reconfigurations to the park area from 06/11/2018 — 06/18/2018

For the time being, we will just put these refurbishments and downtimes into separate variables for later reference. It is beyond the current scope of our project to 'predict' downtimes, and we don't want them to affect the other measures for wait times or to throw off our graphs.

First, let's confirm that these dates match our data.

In [14]:
pirates[pirates['date'] == '2015-06-08']

,date,datetime,wait


It appears that for periods where an attraction is closed, it's not logged in the table at all, which is good; we don't need to account for refurbishments.

Now let's move our downtimes out of our main dataframes.

In [15]:
rides_closed = []
for ride_name, ride in zip(all_ride_names, all_rides):
    ride_closed = ride.copy()[ride['wait'] == -999]
    rides_closed.append([ride_name, ride_closed])

In [16]:
rides_closed[0]

['Seven Dwarfs Mine Train',
              date            datetime   wait
 107    2015-01-01 2015-01-01 20:26:24 -999.0
 108    2015-01-01 2015-01-01 20:58:01 -999.0
 109    2015-01-01 2015-01-01 20:58:26 -999.0
 279    2015-01-02 2015-01-02 23:50:15 -999.0
 374    2015-01-03 2015-01-03 19:02:32 -999.0
 ...           ...                 ...    ...
 321552 2021-12-28 2021-12-28 15:24:32 -999.0
 321616 2021-12-28 2021-12-28 21:41:16 -999.0
 321617 2021-12-28 2021-12-28 21:48:12 -999.0
 321618 2021-12-28 2021-12-28 21:54:09 -999.0
 321619 2021-12-28 2021-12-28 22:00:16 -999.0
 
 [23062 rows x 3 columns]]

In [17]:
def remove_closures(lst):
    for ride in lst:
        ride.drop(ride[ride['wait'] == -999].index, inplace=True)
    return lst
    
remove_closures(all_rides)

for ride_name, ride in zip(all_ride_names, all_rides):
    print(f'{ride_name}:\n'.format(ride_name), ride.shape, '\n')

Seven Dwarfs Mine Train:
 (298569, 3) 

Pirates of the Caribbean:
 (280031, 3) 

Splash Mountain:
 (255910, 3) 

Soarin:
 (269468, 3) 

Spaceship Earth:
 (264441, 3) 

Alien Swirling Saucers:
 (124803, 3) 

Rock-n-Rollercoaster:
 (262508, 3) 

Slinky Dog Dash:
 (125289, 3) 

Toy Story Midway Mania:
 (274585, 3) 

Expedition Everest:
 (255172, 3) 

Avatar Flight of Passage:
 (179492, 3) 

Kilimanjaro Safaris:
 (243752, 3) 

Navi River Journey:
 (174554, 3) 



Now that the downtimes are removed, let's check on our wait time numbers now.

In [18]:
desc_table(all_ride_names, all_rides)

,measures,Seven Dwarfs Mine Train,Pirates of the Caribbean,Splash Mountain,Soarin,Spaceship Earth,Alien Swirling Saucers,Rock-n-Rollercoaster,Slinky Dog Dash,Toy Story Midway Mania,Expedition Everest,Avatar Flight of Passage,Kilimanjaro Safaris,Navi River Journey
0,count,298569.000000,280031.000000,255910.000000,269468.000000,264441.000000,124803.000000,262508.000000,125289.00000,274585.000000,255172.000000,179492.000000,243752.000000,174554.000000
1,mean,75.607893,28.095475,43.239721,44.974991,18.637488,29.674046,58.273931,71.16707,53.531974,31.849023,113.414776,39.484841,61.634319
2,std,173.624079,17.961461,30.241243,27.260359,14.568854,15.969794,31.928949,28.44705,175.015380,22.874179,131.942977,28.534206,32.473301
3,min,-92918.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,25%,50.000000,10.000000,15.000000,25.000000,5.000000,20.000000,35.000000,55.00000,30.000000,15.000000,70.000000,15.000000,40.000000
5,50%,70.000000,25.000000,40.000000,40.000000,15.000000,30.000000,50.000000,70.00000,50.000000,30.000000,110.000000,35.000000,60.000000
6,75%,95.000000,40.000000,65.000000,60.000000,25.000000,40.000000,75.000000,85.00000,70.000000,45.000000,150.000000,55.000000,80.000000
7,max,300.000000,185.000000,300.000000,240.000000,155.000000,190.000000,300.000000,305.00000,90387.000000,180.000000,47897.000000,300.000000,225.000000


It looks like there are still a few major anomalies: Seven Dwarfs has a minimum wait in the negatives, while Toy Story and Flight of Passage have maximum weights in the tens of thousands!

Let's investigate these and see if we can correct them to something more appropriate.

In [19]:
def max_10(all_ride_names, all_rides):
    df_list = []
    for ride_name, ride in zip(all_ride_names, all_rides):
        df_max = pd.DataFrame(ride['wait'].sort_values(ascending=False).head(10).tolist(), columns=[ride_name])
        df_list.append(df_max)
    wdw_max = pd.concat(df_list, axis=1)
    return wdw_max

max_10(all_ride_names, all_rides)

,Seven Dwarfs Mine Train,Pirates of the Caribbean,Splash Mountain,Soarin,Spaceship Earth,Alien Swirling Saucers,Rock-n-Rollercoaster,Slinky Dog Dash,Toy Story Midway Mania,Expedition Everest,Avatar Flight of Passage,Kilimanjaro Safaris,Navi River Journey
0,300.0,185.0,300.0,240.0,155.0,190.0,300.0,305.0,90387.0,180.0,47897.0,300.0,225.0
1,300.0,180.0,300.0,240.0,155.0,190.0,250.0,300.0,658.0,180.0,17134.0,210.0,225.0
2,280.0,135.0,210.0,240.0,155.0,190.0,240.0,295.0,300.0,180.0,4338.0,210.0,225.0
3,280.0,130.0,200.0,240.0,140.0,190.0,230.0,295.0,300.0,180.0,2911.0,210.0,225.0
4,270.0,125.0,200.0,240.0,135.0,190.0,230.0,295.0,300.0,180.0,1171.0,210.0,225.0
5,270.0,125.0,200.0,240.0,135.0,190.0,230.0,295.0,300.0,180.0,390.0,210.0,225.0
6,270.0,125.0,195.0,240.0,135.0,190.0,230.0,295.0,300.0,180.0,390.0,210.0,225.0
7,270.0,125.0,195.0,240.0,135.0,190.0,225.0,295.0,300.0,180.0,360.0,210.0,225.0
8,270.0,125.0,195.0,240.0,135.0,190.0,220.0,295.0,300.0,180.0,360.0,210.0,225.0
9,270.0,125.0,195.0,240.0,135.0,190.0,220.0,295.0,300.0,180.0,360.0,210.0,225.0


In [20]:
def min_10(all_ride_names, all_rides):
    df_list = []
    for ride_name, ride in zip(all_ride_names, all_rides):
        df_max = pd.DataFrame(ride['wait'].sort_values(ascending=True).head(10).tolist(), columns=[ride_name])
        df_list.append(df_max)
    wdw_min = pd.concat(df_list, axis=1)
    return wdw_min

min_10(all_ride_names, all_rides)

,Seven Dwarfs Mine Train,Pirates of the Caribbean,Splash Mountain,Soarin,Spaceship Earth,Alien Swirling Saucers,Rock-n-Rollercoaster,Slinky Dog Dash,Toy Story Midway Mania,Expedition Everest,Avatar Flight of Passage,Kilimanjaro Safaris,Navi River Journey
0,-92918.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


It seems that Seven Dwarfs is the only table with a too-low value, but two tables have too-high outliers:
* Toy Story Midway Mania has two outliers at 90387 and 658
* Avatar Flight of Passage has waits at 47897, 17134, 4338, 2911, and 1171

All other waits seem high, but Walt Disney World has seen very high waits during peak seasons for brand-new attractions in the past, so they are not inconceivable. Let's fix the obvious outliers.

In [21]:
print(dwarfs.loc[dwarfs['wait'] == -92918])
print(toystory.loc[toystory['wait'] == 90387])
print(toystory.loc[toystory['wait'] == 658])
print(fop.loc[fop['wait'] == 47897])
print(fop.loc[fop['wait'] == 17134])
print(fop.loc[fop['wait'] == 4338])
print(fop.loc[fop['wait'] == 2911])
print(fop.loc[fop['wait'] == 1171])

             date            datetime     wait
105109 2017-02-20 2017-02-20 09:04:37 -92918.0
             date            datetime     wait
129139 2018-02-03 2017-12-02 16:28:57  90387.0
             date            datetime   wait
194866 2019-06-25 2019-06-25 09:04:38  658.0
            date            datetime     wait
22777 2017-11-08 2017-10-06 09:34:33  47897.0
            date            datetime     wait
96288 2019-05-27 2019-05-15 19:12:13  17134.0
            date            datetime    wait
35367 2018-02-23 2018-02-20 15:46:37  4338.0
            date            datetime    wait
65745 2018-10-07 2018-10-05 09:41:12  2911.0
            date            datetime    wait
79439 2019-01-23 2019-01-22 21:03:36  1171.0


It also looks like one of the Toy Story Midway Mania outliers and all of the Flight of Passage outliers are also paired with major datetime discrepancies: the `datetime` value is completely different from the `date`! Let's fix all of these.

In [22]:
dwarfs.loc[dwarfs['date'] == '02/20/2017'].head(10)

,date,datetime,wait
105109,2017-02-20,2017-02-20 09:04:37,-92918.0
105110,2017-02-20,2017-02-20 09:07:07,30.0
105111,2017-02-20,2017-02-20 09:14:04,90.0
105112,2017-02-20,2017-02-20 09:21:06,90.0
105113,2017-02-20,2017-02-20 09:27:13,90.0
105114,2017-02-20,2017-02-20 09:35:11,90.0
105115,2017-02-20,2017-02-20 09:42:17,90.0
105116,2017-02-20,2017-02-20 09:49:07,90.0
105117,2017-02-20,2017-02-20 09:53:21,85.0
105118,2017-02-20,2017-02-20 09:56:07,90.0


Magic Kingdom opened at 9:00 am on 02/20/2017, so the Seven Dwarfs wait can be replaced with the time recorded just a few minutes later, 30 minutes.

In [23]:
dwarfs.loc[dwarfs['wait'] == -92918, ['wait']] = 30

Both the date and the time in these records' `datetime` column are so incorrect that it would ultimately be easier to simply remove them.

In [26]:
toystory.drop(toystory[toystory['wait'] == 90387].index, inplace=True)
toystory.drop(toystory[toystory['wait'] == 658].index, inplace=True)
fop.drop(fop[fop['wait'] == 47897].index, inplace=True)
fop.drop(fop[fop['wait'] == 17134].index, inplace=True)
fop.drop(fop[fop['wait'] == 4338].index, inplace=True)
fop.drop(fop[fop['wait'] == 2911].index, inplace=True)
fop.drop(fop[fop['wait'] == 1171].index, inplace=True)

In [27]:
desc_table(all_ride_names, all_rides)

,measures,Seven Dwarfs Mine Train,Pirates of the Caribbean,Splash Mountain,Soarin,Spaceship Earth,Alien Swirling Saucers,Rock-n-Rollercoaster,Slinky Dog Dash,Toy Story Midway Mania,Expedition Everest,Avatar Flight of Passage,Kilimanjaro Safaris,Navi River Journey
0,count,298569.000000,280031.000000,255910.000000,269468.000000,264441.000000,124803.000000,262508.000000,125289.00000,274583.000000,255172.000000,179487.000000,243752.000000,174554.000000
1,mean,75.919205,28.095475,43.239721,44.974991,18.637488,29.674046,58.273931,71.16707,53.200788,31.849023,113.008708,39.484841,61.634319
2,std,34.364771,17.961461,30.241243,27.260359,14.568854,15.969794,31.928949,28.44705,30.179034,22.874179,54.081216,28.534206,32.473301
3,min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
4,25%,50.000000,10.000000,15.000000,25.000000,5.000000,20.000000,35.000000,55.00000,30.000000,15.000000,70.000000,15.000000,40.000000
5,50%,70.000000,25.000000,40.000000,40.000000,15.000000,30.000000,50.000000,70.00000,50.000000,30.000000,110.000000,35.000000,60.000000
6,75%,95.000000,40.000000,65.000000,60.000000,25.000000,40.000000,75.000000,85.00000,70.000000,45.000000,150.000000,55.000000,80.000000
7,max,300.000000,185.000000,300.000000,240.000000,155.000000,190.000000,300.000000,305.00000,300.000000,180.000000,390.000000,300.000000,225.000000


Much better!

## Adding Park and Ride Labels

Our ride tables seem nicely cleaned now, but it might be useful to add columns labeling their name and the park they're each located in, in case these tables get combined in some way later in our process.

In [28]:
def park_from_ride(row):
    if row['ride'] in ['Seven Dwarfs Mine Train', 'Pirates of the Caribbean', 'Splash Mountain']:
        return 'Magic Kingdom'
    elif row['ride'] in ['Soarin', 'Spaceship Earth']:
        return 'Epcot'
    elif row['ride'] in ['Alien Swirling Saucers', 'Rock-n-Rollercoaster', 'Slinky Dog Dash', 'Toy Story Midway Mania']:
        return 'Hollywood Studios'
    else:
        return 'Animal Kingdom'

In [29]:
for ride_name, ride in zip(all_ride_names, all_rides):
        ride['ride'] = ride_name
        ride['park'] = ride.apply(lambda row: park_from_ride(row), axis=1)

Now let's try combining ALL of our park tables into one large dataframe we can use for our higher level visualizations later.

In [48]:
ride_copy_list = []    
for ride_name, ride in zip(all_ride_names, all_rides):
        ride_copy = ride.copy()
        ride_copy_list.append(ride_copy)
        
all_df = pd.concat(ride_copy_list)
all_df.head()

,park,ride,date,datetime,wait
0,Magic Kingdom,Seven Dwarfs Mine Train,2015-01-01,2015-01-01 07:51:12,45.0
1,Magic Kingdom,Seven Dwarfs Mine Train,2015-01-01,2015-01-01 08:02:13,60.0
2,Magic Kingdom,Seven Dwarfs Mine Train,2015-01-01,2015-01-01 08:05:30,54.0
3,Magic Kingdom,Seven Dwarfs Mine Train,2015-01-01,2015-01-01 08:09:12,60.0
4,Magic Kingdom,Seven Dwarfs Mine Train,2015-01-01,2015-01-01 08:16:12,60.0


In [49]:
print(all_df['ride'].unique())
print(all_df['park'].unique())

['Seven Dwarfs Mine Train' 'Pirates of the Caribbean' 'Splash Mountain'
 'Soarin' 'Spaceship Earth' 'Alien Swirling Saucers'
 'Rock-n-Rollercoaster' 'Slinky Dog Dash' 'Toy Story Midway Mania'
 'Expedition Everest' 'Avatar Flight of Passage' 'Kilimanjaro Safaris'
 'Navi River Journey']
['Magic Kingdom' 'Epcot' 'Hollywood Studios' 'Animal Kingdom']


Everything looks good! Let's export our cleaned dataframes to CSV and begin the next step.

In [100]:
# dwarfs.to_csv('dwarfs_clean.csv', index=False)
# aliens.to_csv('alien_saucers_clean.csv', index=False)
# dinosaur.to_csv('dinosaur_clean.csv', index=False)
# everest.to_csv('expedition_everest_clean.csv', index=False)
# fop.to_csv('flight_of_passage_clean.csv', index=False)
# safari.to_csv('kilimanjaro_safaris_clean.csv', index=False)
# river.to_csv('navi_river_clean.csv', index=False)
# pirates.to_csv('pirates_of_caribbean_clean.csv', index=False)
# rockcoaster.to_csv('rock_n_rollercoaster_clean.csv', index=False)
# slinky.to_csv('slinky_dog_clean.csv', index=False)
# soarin.to_csv('soarin_clean.csv', index=False)
# spaceship.to_csv('spaceship_earth_clean.csv', index=False)
# splash.to_csv('splash_mountain_clean.csv', index=False)
# toystory.to_csv('toy_story_mania_clean.csv', index=False)
# all_df.to_csv('all_df.csv', index=False)